# **Fast Forward Neural Network** (FFNN) Implementation
#### _with **Forward Propagation**_

#### Made by:
- Samuel Christoper Swandi - 13520075
- Grace Claudia - 13520078
- Ubaidillah Ariq Prathama - 13520085
- Patrick Amadeus Irawan - 13520109

------------

List of Content

1. [Library & Dependencies](#library)
2. [Helper Function](#helper)
3. [Neural Network Visualization](#visualization)

## Library & Dependencies <a name="library"></a>

In [ ]:
!pip install pyvis==0.3.2
!pip install networkx==2.6.3
!pip install numpy==1.21.6

In [56]:
class ConnectedLayer:
    def __init__(self, input_size, output_size, weights):
        self.input_size = input_size
        self.output_size = output_size
        self.weights = weights[1:]
        self.bias = weights[0]

    def forward(self, input):
        self.input = input
        return np.dot(input, self.weights) + self.bias

#### Activation Layer

In [57]:
class ActivationLayer:
    def __init__(self, activation):
        self.activation = activation
    
    def forward(self, input):
        self.input = input
        return self.activation(input)

#### Neural Network

In [58]:
class Network:
    def __init__(self):
        self.layers = []

    def add(self, layer):
        self.layers.append(layer)
    
    def predict(self, input_data):
        samples = len(input_data)
        result = []

        for i in range(samples):
            output = input_data[i][1:]
            for layer in self.layers:
                output = layer.forward(output)
            result.append(output)

        return result


## Model Load Section

In [59]:
TEST_FOLDER = '../test/'
# read file from test folder
def read_file(file_name):
    with open(TEST_FOLDER+file_name, 'r') as file:
        data = [i.rstrip("\n") for i in file.readlines()]
    return data

data = read_file('xor.txt')

## Loading Weights

In [63]:
idx = 1
weights = []

for i in range(int(data[0]) - 1):
    prev_size, layer_size, activation = [int(i) for i in data[idx].split()]
    
    idx += 1
    weight = []
    for j in range(prev_size + 1):
        weight.append([int(i) for i in data[idx].split()])
        idx += 1
    
    weights.append(weight)

In [64]:
x_train = np.array([[1,0,0], [1,0,1], [1,1,0], [1,1,1]])
net = Network()

net.add(ConnectedLayer(2, 2, weights[0]))
net.add(ActivationLayer(sigmoid))
net.add(ConnectedLayer(2, 1, weights[1]))
net.add(ActivationLayer(sigmoid))

out = net.predict(x_train)
out

[array([4.54391049e-05]),
 array([0.99995452]),
 array([0.99995452]),
 array([4.54391049e-05])]

In [ ]:
import numpy as np
import os

from pyvis.network import Network

## **Helper** Function <a class="anchor" id="helper"></a>

#### Activation Function

In [ ]:
def linear(net):
    return net

def ReLU(net):
    return np.maximum(0,net)

def sigmoid(net):
    return 1/(1+np.exp(-net))

def softmax(net):
    return np.exp(net)/np.sum(np.exp(net))

#### Loss Function

In [ ]:
def mse(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

def sse(y_true, y_pred):
    return np.sum(np.square(y_true - y_pred))

# Classes

#### Connected Layer

In [ ]:
n = int(data[0])
act = {0: "linear", 1: "ReLU", 2: "sigmoid", 3: "softmax"}
idx = 1

XSTEP = 300
YSTEP = 300
SIZE = 10

nodes = []
node_i = 1
value = []
x_val = 0
y_val = 0
x = []
y = []
label = []
color = []
edge = []

src_idx = 0

for i_layer in range(n - 1):
    n_curr, n_next, act = [int(i) for i in data[idx].split()]

    if i_layer == 0:  # means that this is the first layer, hence construct input
        for i_node in range(n_curr + 1):
            if i_node == 0:  # bias
                color.append("#dd4b39")
                label.append("Input[bias]")
            else:
                color.append("#162347")
                label.append("Input[{}]".format(i_node))
            value.append(SIZE)
            x.append(x_val)
            y.append(y_val)
            y_val += YSTEP
            nodes.append(node_i)
            node_i += 1
        x_val += XSTEP

    y_val = 0
    # always construct the next layer
    for i_node in range(n_next + 1):
        if i_node == 0:
            if i_layer == n - 2:
                continue
            color.append("#dd4b39")
            label.append("HL{}[bias]".format(i_layer + 1))
        else:
            color.append("#162347")
            if i_layer == n - 2:
                label.append("Output[{}]".format(i_node))
            else:
                label.append("HL{}[{}]".format(i_layer + 1, i_node))
        value.append(SIZE)
        x.append(x_val)
        y.append(y_val)
        y_val += YSTEP
        nodes.append(node_i)
        node_i += 1
    x_val += XSTEP

    idx += 1
    for origin in range(n_curr + 1):
        dst_idx = -1
        for w in reversed(data[idx].split()):
            edge.append((nodes[src_idx], nodes[dst_idx], w))
            dst_idx -= 1
        src_idx += 1
        idx += 1


In [ ]:
g = Network(notebook=True, cdn_resources="remote")
g.add_nodes(
	nodes,
    value = value,
    x=x,y=y,
    label = label,
    color = color
)

for e in edge:
    g.add_edge(e[0], e[1], title = e[2], color="#162347")

for n in g.nodes:
    n.update({'physics': False})
    
g.show("./tmp/network.html")